In [11]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [132]:
from mvl import genData
from torch import tensor

liabParams55cov = genData.genParams(pis=tensor([.1, .1, .05]), rrMeans=tensor([3., 2.]), afMean = tensor(1e-4), pDs = tensor([.01, .01]), afShape=tensor(50.), nCases=tensor([1.5e4, 1.5e4, 4e3]), nCtrls=tensor(5e4), covShared=tensor([ [1, .95], [.95, 1] ]), meanEffectCovarianceScale=tensor(.01))[0]
liabParams55cov["pDs"] = liabParams55cov["pDs"][0:2]
liabData55cov = genData.v6liability(**liabParams55cov)
liabParams55cov["pDs"] = liabData55cov["PDs"] #TODO: normalized names, and indicate that the params pDs are incomplete

, 1.8941e-05, 5.0509e-06], dtype=torch.float64)
19768 p tensor([4.5868e-05, 1.3760e-05, 1.3760e-05, 3.6694e-06], dtype=torch.float64)
19769 p tensor([5.2020e-05, 1.5606e-05, 1.5606e-05, 4.1616e-06], dtype=torch.float64)
19770 p tensor([7.3299e-05, 2.1990e-05, 2.1990e-05, 5.8639e-06], dtype=torch.float64)
19771 p tensor([5.7161e-05, 1.7148e-05, 1.7148e-05, 4.5729e-06], dtype=torch.float64)
19772 p tensor([5.1760e-05, 1.5528e-05, 1.5528e-05, 4.1408e-06], dtype=torch.float64)
19773 p tensor([5.0661e-05, 5.9077e-05, 1.5939e-05, 1.5754e-05], dtype=torch.float64)
19774 p tensor([7.5587e-05, 2.2676e-05, 2.2676e-05, 6.0469e-06], dtype=torch.float64)
19775 p tensor([6.5469e-05, 1.9641e-05, 1.9641e-05, 5.2375e-06], dtype=torch.float64)
19776 p tensor([6.3796e-05, 1.9139e-05, 1.9139e-05, 5.1037e-06], dtype=torch.float64)
19777 p tensor([6.4408e-05, 1.9322e-05, 1.9322e-05, 5.1526e-06], dtype=torch.float64)
19778 p tensor([5.6438e-05, 6.1244e-05, 4.0764e-05, 1.3673e-05], dtype=torch.float64)
19779 

In [130]:
import torch
p_test = tensor([6.8854e-05, 2.0656e-05, 2.0656e-05, 5.5083e-06])
print(p_test.sum())
total_af_mean = 1e-4

mn = td.Multinomial(probs=p_test, total_count= scaled_N)

# OK, so we can recover the exact
print("mn.probs", mn.probs)
print("p_test[0] / mn.probs[0])", tensor(p_test[0] / mn.probs[0], dtype=torch.float64))
print("mn.probs * (p_test[0] / mn.probs[0])", mn.probs * (p_test[0] / mn.probs[0]))
# This works perfectly
print("mn.probs * total_af", mn.probs * total_af)
print("mn.mean", mn.mean)

tensor(0.0001)
mn.probs tensor([0.5952, 0.1786, 0.1786, 0.0476])
p_test[0] / mn.probs[0]) tensor(0.0001, dtype=torch.float64)
mn.probs * (p_test[0] / mn.probs[0]) tensor([5.9524e-05, 1.7857e-05, 1.7857e-05, 4.7619e-06])
mn.probs * total_af tensor([6.3168e-05, 1.8950e-05, 1.8950e-05, 5.0534e-06])
mn.mean tensor([2.9762, 0.8929, 0.8929, 0.2381])


In [115]:
test_probs = tensor([.0005, .0006, .0002])
print(test_probs.sum())
# 3.89 is .0005 / the actual probability in position 1
# .0005/.3846 == .0013
# This is also the test_probs sum
td.Multinomial(probs=tensor([.0005, .0006, .0002]), total_count= scaled_N).probs * test_probs.sum()

tensor(0.0013)


tensor([0.0005, 0.0006, 0.0002])

In [73]:
.005 

tensor([2.3359, 0.7189, 1.7773, 0.4740])

In [74]:
td.Multinomial(probs=p_test, total_count= scaled_N).param_shape

torch.Size([4])

In [3]:
liabParams55cov["pDs"]

tensor([0.0100, 0.0100, 0.0067], dtype=torch.float64)

In [9]:
liabParams55cov["pDsample"]

KeyError: 'pDsample'

In [217]:
import numpyro
from numpyro.distributions import Multinomial, Beta, Dirichlet, Gamma, Beta, Categorical, Uniform, MultivariateNormal, Normal, LogNormal, Exponential, HalfCauchy, LKJCholesky
from jax import random
from numpyro.infer import MCMC, NUTS
import numpy as np
import jax.numpy as jnp
import jax

numpyro.set_host_device_count(6)
numpyro.enable_x64()

def mix_weights(beta: jnp.array):
    beta_cumprod = (1 - beta).cumprod(-1)
    return jnp.pad(beta, (0,1), constant_values=1) * jnp.pad(beta_cumprod, (1,0), constant_values = 1)


# mu_exp, var_exp = get_log_params(liabParams55cov["afMean"].numpy(), 1)

nHypotheses = 6
kConditions = 4
altCounts = liabData55cov["altCounts"].numpy()
N = len(liabData55cov["altCounts"])

nCases = liabParams55cov["nCases"].numpy()
nCtrls = liabParams55cov["nCtrls"].numpy()
empiricalAfs = altCounts.sum(1) / (nCases.sum() + nCtrls)
print("empirical afs", altCounts / (nCases.sum() + nCtrls))
samplePDs = nCases / (nCases.sum() + nCtrls)
pdsAll = np.array([1 - samplePDs.sum(), *samplePDs])
print("pdsAll", pdsAll)

# TODO: do this in numpy natively
pdsAllShaped = jnp.asarray(tensor(pdsAll).expand(nHypotheses, kConditions).numpy())
pdsAllnp = [1 - samplePDs.sum(), *samplePDs]
pdsAll = jnp.asarray([1 - samplePDs.sum(), *samplePDs])
print(pdsAllShaped)
print(pdsAll.shape)
print((1/pdsAll).shape)
alpha = .01
print("nHypotheses", nHypotheses)

component_af_means = jnp.array([.005, .006, .006, .013])
exponential_prior = jnp.array([.01]).repeat(kConditions)

def model(data):
    # This also works, for a single set of shraed parameters
    # with numpyro.plate("conc_plate", 1):
    #     conc = numpyro.sample('conc', Exponential(.001))

    with numpyro.plate("beta_plate", nHypotheses-1):
        beta = numpyro.sample("beta", Beta(1, alpha))

    with numpyro.plate("prob_plate", nHypotheses):
        conc = numpyro.sample('conc', Exponential(exponential_prior).to_event(1))
        probs = numpyro.sample("probs", Dirichlet(conc)) * pdsAll

    with numpyro.plate("data", N):
        z = numpyro.sample("z", Categorical(mix_weights(beta)))
        return numpyro.sample("obs", Multinomial(probs=probs[z]), obs=data)

def modelUniform(data):
    # This also works, for a single set of shraed parameters
    # with numpyro.plate("conc_plate", 1):
    #     conc = numpyro.sample('conc', Exponential(.001))

    with numpyro.plate("beta_plate", nHypotheses-1):
        beta = numpyro.sample("beta", Beta(1, alpha))

    with numpyro.plate("prob_plate", nHypotheses):
        conc = numpyro.sample('conc', Uniform(1, 25000).to_event(1))
        probs = numpyro.sample("probs", Dirichlet(conc)) * pdsAll

    with numpyro.plate("data", N):
        z = numpyro.sample("z", Categorical(mix_weights(beta)))
        return numpyro.sample("obs", Multinomial(probs=probs[z]), obs=data)

def model1(data):
    # This also works, for a single set of shraed parameters
    with numpyro.plate("conc_plate", kConditions):
        conc = numpyro.sample('conc', Exponential(.01, validate_args=False))

    with numpyro.plate("beta_plate", nHypotheses-1):
        beta = numpyro.sample("beta", Beta(1, alpha, validate_args=False))

    with numpyro.plate("prob_plate", nHypotheses):
        probs = numpyro.sample("probs", Dirichlet(conc, validate_args=False)) * pdsAll

    with numpyro.plate("data", N):
        z = numpyro.sample("z", Categorical(mix_weights(beta)))
        return numpyro.sample("obs", Multinomial(probs=probs[z], validate_args=False), obs=data)

def modelLogNormal(data):
    # with numpyro.plate("prob_plate", kConditions):
    #      # Vector of variances for each of the d variables
    #     theta = numpyro.sample("theta", HalfCauchy(jnp.ones(kConditions)).to_event(1))
    #     print("theta", theta)
    #     # Lower cholesky factor of a correlation matrix
    #     eta = jnp.ones(1)  # Implies a uniform distribution over correlation matrices
    #     L_omega = numpyro.sample("L_omega", LKJCholesky(kConditions, eta))
    #     # Lower cholesky factor of the covariance matrix
    #     L_Omega = jnp.matmul(jnp.diag(jnp.sqrt(theta)), L_omega)
    #     print("L_Omega", L_Omega)
    # For inference with SVI, one might prefer to use torch.bmm(theta.sqrt().diag_embed(), L_omega)

        # Vector of expectations
        # mu = jnp.zeros(kConditions)
        # print("L_Omega", L_Omega, L_Omega.shape)

    with numpyro.plate("beta_plate", nHypotheses-1):
        beta = numpyro.sample("beta", Beta(1, alpha))

    with numpyro.plate("prob_plate", nHypotheses):
        # print("pdsAll", pdsAll)
        # theta = numpyro.sample("theta", HalfCauchy(jnp.ones(kConditions)).to_event(1))
        # print("theta", theta)
        # Lower cholesky factor of a correlation matrix
        eta = jnp.ones(1)  # Implies a uniform distribution over correlation matrices
        L_omega = numpyro.sample("L_omega", LKJCholesky(kConditions, eta))
        # Lower cholesky factor of the covariance matrix
        # L_Omega = jnp.matmul(jnp.diag(jnp.sqrt(theta)), L_omega)
        # print("L_Omega", L_Omega)
    # For inference with SVI, one might pr
        probs = jax.nn.softmax(numpyro.sample("probs", MultivariateNormal(pdsAll, L_omega ) ) )

    with numpyro.plate("data", N):
        z = numpyro.sample("z", Categorical(mix_weights(beta)))
        return numpyro.sample("obs", Multinomial(probs=probs[z]), obs=data)

empirical afs [[2.38095238e-05 3.57142857e-05 2.38095238e-05 1.19047619e-05]
 [2.38095238e-05 3.57142857e-05 8.33333333e-05 1.19047619e-05]
 [3.57142857e-05 0.00000000e+00 3.57142857e-05 0.00000000e+00]
 ...
 [3.57142857e-05 0.00000000e+00 3.57142857e-05 1.19047619e-05]
 [3.57142857e-05 1.19047619e-05 4.76190476e-05 0.00000000e+00]
 [4.76190476e-05 3.57142857e-05 1.19047619e-05 0.00000000e+00]]
pdsAll [0.59523809 0.17857143 0.17857143 0.04761905]
[[0.59523809 0.17857143 0.17857143 0.04761905]
 [0.59523809 0.17857143 0.17857143 0.04761905]
 [0.59523809 0.17857143 0.17857143 0.04761905]
 [0.59523809 0.17857143 0.17857143 0.04761905]
 [0.59523809 0.17857143 0.17857143 0.04761905]
 [0.59523809 0.17857143 0.17857143 0.04761905]]
(4,)
(4,)
nHypotheses 6


In [24]:
Multinomial?

Signature: Multinomial(total_count=1, probs=None, logits=None, validate_args=None)
Docstring: <no docstring>
File:      ~/miniconda3/lib/python3.7/site-packages/numpyro/distributions/discrete.py
Type:      function


In [ ]:
def fn():
x = nu'mpyro.sample('x', numpyro.distributions.Exponential(.01))
    return x
   
print(numpyro.handlers.seed(fn, rng_seed=2)())

In [ ]:
jnp.eye(kConditions)

In [ ]:
init_rng_key = random.PRNGKey(12273)
mcmcLN = MCMC(NUTS(modelLogNormal), 100, 1000, jit_model_args=True)
mcmcLN.run(init_rng_key, altCounts)
mcmcLN.print_summary()

In [ ]:
\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\]
];;;;'///////////'

In [ ]:
# Scaling by pDs gives absolutely no difference in concentrations or probabilities.
sLN = mcmcLN.get_samples()["probs"]
print(sLN.mean(0))
print("jax.nn.softmax", jax.nn.softmax(sLN.mean(0)))

# concs = mcmcAf.get_samples()["conc"]
# print("conc std", concs.std(0))
# print("conc mean", concs.mean(0))
# print("DIrichlet version", Dirichlet(concs.mean(0)).mean)

betaScaledLNInDirichlet = mcmcLN.get_samples()['beta']
print("inferred stick-breaking weights", mix_weights(betaScaledLNInDirichlet).mean(0))

In [216]:

init_rng_key = random.PRNGKey(12273)
mcmcAf1 = MCMC(NUTS(model1), 100, 1000)
mcmcAf1.run(init_rng_key, altCounts)
mcmcAf1.print_summary()

ValueError: Invalid shape: expected (), actual (1,)

In [ ]:
# 1/3rd as fast as not having deterministic
# this runs after mcmc fits, not during
def postprocess(arg):
    print("postprocess arg", arg)
    return arg
init_rng_key = random.PRNGKey(12273)
mcmcAf = MCMC(NUTS(model), 100, 1000, jit_model_args=True, chain_method='vectorized')
mcmcAf.run(init_rng_key, altCounts)
mcmcAf.print_summary()

In [136]:
# 1/3rd as fast as not having deterministic
# this runs after mcmc fits, not during
def postprocess(arg):
    print("postprocess arg", arg)
    return arg
init_rng_key = random.PRNGKey(12273)
mcmcAf = MCMC(NUTS(model), 100, 1000, jit_model_args=True, chain_method='vectorized')
mcmcAf.run(init_rng_key, altCounts)
mcmcAf.print_summary()

sample: 100%|██████████| 1100/1100 [1:09:18<00:00,  3.78s/it, 1023 steps of size 2.59e-03. acc. prob=0.96]

                mean       std    median      5.0%     95.0%     n_eff     r_hat
   beta[0]      0.06      0.02      0.06      0.04      0.08    182.37      1.02
   beta[1]      0.06      0.02      0.06      0.03      0.09    554.89      1.00
   beta[2]      0.84      0.02      0.84      0.82      0.87    237.63      1.01
   beta[3]      1.00      0.02      1.00      1.00      1.00     82.88      1.01
   beta[4]      0.97      0.12      1.00      0.95      1.00    578.65      1.00
 conc[0,0]  13038.64   6610.88  12191.86   3517.39  22582.26    238.56      1.00
 conc[0,1]  13208.51   6665.31  12331.29   3197.41  22323.45    214.97      1.00
 conc[0,2]  11838.03   6013.15  10940.42   2562.97  19598.51    236.72      1.00
 conc[0,3]   3742.90   1910.92   3465.25    883.32   6308.56    222.31      1.00
 conc[1,0]  18195.71  10058.86  16654.11   1962.22  31259.45    927.89      1.00
 

In [137]:
# Scaling by pDs gives absolutely no difference in concentrations or probabilities.
sAf = mcmcAf.get_samples()["probs"]
print(sAf.mean(0))

concs = mcmcAf.get_samples()["conc"]
print("conc std", concs.std(0))
print("conc mean", concs.mean(0))
print("DIrichlet version", Dirichlet(concs.mean(0)).mean)

betaScaledInDirichlet = mcmcAf.get_samples()['beta']
print("inferred stick-breaking weights", mix_weights(betaScaledInDirichlet).mean(0))

[[0.31155055 0.31563388 0.2833775  0.08943807]
 [0.46183678 0.10544625 0.33964039 0.09307659]
 [0.59777092 0.17466671 0.18151134 0.04605103]
 [0.39448879 0.38222496 0.12336955 0.0999167 ]
 [0.23897428 0.27588011 0.2562659  0.2288797 ]
 [0.26062323 0.26228489 0.22314267 0.25394921]]
conc std [[ 6607.57696014  6661.97867602  6010.14543301  1909.96406331]
 [10053.83082937  2419.63313846  7320.52119931  2023.88433427]
 [11853.49602242  3476.25666153  3578.84006459   916.64961115]
 [ 7723.04756261  7409.86714954  2428.93670449  1943.97564794]
 [ 9018.78832794 11100.42135148 11267.40362621  9622.80823251]
 [10103.11212855  9817.60443359  9357.68850064  9291.70836826]]
conc mean [[13038.64192388 13208.50890876 11838.02514843  3742.89787881]
 [18195.70660022  4165.48051301 13379.17702105  3662.74881349]
 [23536.36083504  6880.98470871  7142.98853037  1814.32572156]
 [16149.28844425 15632.8207465   5046.6974647   4087.32079326]
 [ 9535.67677016 11294.97619353 10491.67767673  9223.83084012]
 [10

In [204]:
broadcastedEmpiricalAfs = np.broadcast_to(np.array([[empiricalAfs]]).T, (20_000, 6, 4))
sAfscaled = (sAf.mean(0) * broadcastedEmpiricalAfs).mean(0)

sAfscaled2 = (sAf.mean(0) * liabParams55cov["afMean"].numpy())
print("sAfscaled2", sAfscaled2)
print("sAfscaled1", np.asarray(sAfscaled))
print(np.asarray(sAfscaled) / liabParams55cov["afMean"].numpy())

sAfscaled2 [[3.11550542e-05 3.15633871e-05 2.83377498e-05 8.94380640e-06]
 [4.61836770e-05 1.05446243e-05 3.39640379e-05 9.30765828e-06]
 [5.97770904e-05 1.74666708e-05 1.81511334e-05 4.60510279e-06]
 [3.94488785e-05 3.82224950e-05 1.23369545e-05 9.99166950e-06]
 [2.38974275e-05 2.75880108e-05 2.56265896e-05 2.28879696e-05]
 [2.60623223e-05 2.62284880e-05 2.23142666e-05 2.53949206e-05]]
sAfscaled1 [[3.22753388e-05 3.26983548e-05 2.93567288e-05 9.26541100e-06]
 [4.78443663e-05 1.09237917e-05 3.51853290e-05 9.64234640e-06]
 [6.19265766e-05 1.80947437e-05 1.88038184e-05 4.77069473e-06]
 [4.08673954e-05 3.95969132e-05 1.27805711e-05 1.03509535e-05]
 [2.47567398e-05 2.85800304e-05 2.65480797e-05 2.37109834e-05]
 [2.69994806e-05 2.71716214e-05 2.31166511e-05 2.63080802e-05]]
[[0.3227534  0.32698356 0.29356729 0.09265411]
 [0.47844367 0.10923792 0.3518533  0.09642347]
 [0.61926578 0.18094744 0.18803819 0.04770695]
 [0.40867396 0.39596914 0.12780571 0.10350954]
 [0.2475674  0.28580031 0.265480

In [187]:
liabData55cov["afs"][liabData55cov["unaffectedGenes"]].mean(0) / liabParams55cov["afMean"]

tensor([0.5940, 0.1782, 0.1782, 0.0475], dtype=torch.float64)

In [188]:
pdv = (liabData55cov["afs"][liabData55cov["unaffectedGenes"]] / np.array([empiricalAfs[liabData55cov["unaffectedGenes"]]]).T).mean(0)
print("pdv", pdv)
print("pdv scaled", pdv/pdv.sum())

pdv tensor([0.6350, 0.1905, 0.1905, 0.0508], dtype=torch.float64)
pdv scaled tensor([0.5952, 0.1786, 0.1786, 0.0476], dtype=torch.float64)


In [191]:
pdv1 = (liabData55cov["afs"][liabData55cov["affectedGenes"][0]]).mean(0) / liabParams55cov["afMean"]
pdv2 = (liabData55cov["afs"][liabData55cov["affectedGenes"][0]] / np.array([empiricalAfs
[liabData55cov["affectedGenes"][0]]]).T).mean(0)
afAffected0Mean = (liabData55cov["afs"][liabData55cov["affectedGenes"][0]]).mean(0) 
print("pdv1", pdv1)# PD|V
print("pdv2", pdv2)# PD|V
print("afAffected0Mean", afAffected0Mean)
print("pdv1 scaled", pdv1/pdv1.sum())
print("pdv2 scaled", pdv2/pdv2.sum())

pdv1 tensor([0.5702, 0.5480, 0.1774, 0.1461], dtype=torch.float64)
pdv2 tensor([0.4184, 0.3923, 0.1300, 0.1046], dtype=torch.float64)
afAffected0Mean tensor([5.7023e-05, 5.4799e-05, 1.7743e-05, 1.4613e-05], dtype=torch.float64)
pdv1 scaled tensor([0.3955, 0.3801, 0.1231, 0.1014], dtype=torch.float64)
pdv2 scaled tensor([0.4003, 0.3753, 0.1244, 0.1001], dtype=torch.float64)


In [251]:
pdv = (liabData55cov["afs"][liabData55cov["affectedGenes"][1]]).mean(0) / liabParams55cov["afMean"]# PD|V
print("pdv", pdv)
print("pdv scaled", pdv/pdv.sum())

pdv tensor([0.5836, 0.1784, 0.3703, 0.0987], dtype=torch.float64)
pdv scaled tensor([0.4741, 0.1449, 0.3008, 0.0802], dtype=torch.float64)


In [252]:
pdv = (liabData55cov["afs"][liabData55cov["affectedGenes"][2]]).mean(0) / liabParams55cov["afMean"]# PD|V
print("pdv", pdv)
print("pdv scaled", pdv/pdv.sum())

pdv tensor([0.5717, 0.5500, 0.3674, 0.1223], dtype=torch.float64)
pdv scaled tensor([0.3548, 0.3413, 0.2280, 0.0759], dtype=torch.float64)


In [79]:
# [[0.01911611 0.01343591 0.01555622 0.00563713]
#  [0.00406165 0.00276914 0.00327851 0.00133072]
#  [0.01090826 0.00992517 0.0113952  0.00424665]
#  [0.01858757 0.02309512 0.02210252 0.01013227]]
# [[0.47631553 0.10743626 0.26105177 0.03134572]
#  [0.60844487 0.14429386 0.14502415 0.01706672]
#  [0.40694135 0.31180432 0.09986749 0.0380971 ]
#  [0.35437214 0.26981634 0.2043957  0.02945131]]
# conc std [ 12.166306  71.86332   67.61688  157.65123 ]
# conc mean [ 36.537464 213.60443  190.46646  793.18304 ]
# DIrichlet version [0.02961397 0.17312849 0.15437493 0.64288265]
# inferred stick-breaking weights [0.11452598 0.665086   0.13667543 0.08672265]

In [80]:
pdvpv = (liabData55cov["afs"][liabData55cov["affectedGenes"][2]]).mean(0) #/ liabParams55cov["afMean"]# PD|V
pdv = pdvpv / liabParams55cov["afMean"]
print("pdv", pdv)
print("pdvpv", pdvpv)
print("pdvpv.sum()", pdvpv.sum())
# This matches our DM results perfectly; so the DM result matches scale
# how to allow our pdv values to work ok...
print("pdvpv scaled", pdvpv/pdvpv.sum())


pdv tensor([0.5717, 0.5500, 0.3674, 0.1223], dtype=torch.float64)
pdvpv tensor([5.7167e-05, 5.5000e-05, 3.6744e-05, 1.2231e-05], dtype=torch.float64)
pdvpv.sum() tensor(0.0002, dtype=torch.float64)
pdvpv scaled tensor([0.3548, 0.3413, 0.2280, 0.0759], dtype=torch.float64)


In [81]:
liabData55cov["altCounts"][liabData55cov["affectedGenes"][2]]

tensor([[4., 3., 3., 0.],
        [8., 3., 0., 1.],
        [6., 3., 2., 0.],
        ...,
        [5., 8., 1., 1.],
        [3., 3., 5., 2.],
        [6., 5., 2., 0.]], dtype=torch.float64)

In [82]:
pdv.sum()

tensor(1.6114, dtype=torch.float64)

In [83]:
(liabData55cov["afs"][liabData55cov["affectedGenes"][2]]).mean(0) 

tensor([5.7167e-05, 5.5000e-05, 3.6744e-05, 1.2231e-05], dtype=torch.float64)

In [84]:
liabData55cov["afs"][liabData55cov["affectedGenes"][0]].mean(0) 

tensor([5.7375e-05, 5.4857e-05, 1.7848e-05, 1.4628e-05], dtype=torch.float64)

In [85]:
pdsAllnp

[0.5952380895614624, 0.17857143, 0.17857143, 0.04761905]

In [86]:
liabParams55cov["afMean"]

tensor(1.0000e-04)

In [87]:
liabData55cov["afs"][liabData55cov["affectedGenes"][0]].mean(0)

tensor([5.7375e-05, 5.4857e-05, 1.7848e-05, 1.4628e-05], dtype=torch.float64)

In [88]:
liabParams55cov["afMean"]

tensor(1.0000e-04)

In [ ]:
# (1000, 4, 4)
# [[0.00514663 0.00313207 0.00355292 0.00149964]
#  [0.01928454 0.04950465 0.04956334 0.0065191 ]
#  [0.02503735 0.04772532 0.03840724 0.00905628]
#  [0.06047119 0.05520838 0.05940869 0.03415463]]
# [[0.607701   0.1456876  0.14313476 0.01758165]
#  [0.45891437 0.14094748 0.24490298 0.03002501]
#  [0.45052725 0.23326729 0.15602364 0.03166293]
#  [0.4925649  0.1685785  0.18608962 0.03147438]]
# conc std [18.310623   7.338482   7.672899   2.7556536]
# conc mean [46.069946 18.652845 19.728569  6.812487]
# DIrichlet version [0.50479954 0.20438373 0.2161707  0.07464606]
# inferred stick-breaking weights [0.6861298  0.14098327 0.1566307  0.01916244]